# Injection week5

### Instalization

In [10]:
! pip install -q 'cpnest'

In [12]:
! pip install --upgrade cpnest

In [2]:
#!/usr/bin/env python
"""
Tutorial to demonstrate running parameter estimation on a full 15 parameter
space for an injected cbc signal. This is the standard injection analysis script
one can modify for the study of injected CBC events.
"""
import numpy as np
import bilby

# Set the duration and sampling frequency of the data segment that we're
# going to inject the signal into
duration = 4.
sampling_frequency = 2048.

# Specify the output directory and the name of the simulation.
outdir = 'outdir'
label = 'full_15_parameters'
bilby.core.utils.setup_logger(outdir=outdir, label=label)

### Data Setup

In [3]:
# Set up a random seed for result reproducibility.  This is optional!
np.random.seed(88170235)

# We are going to inject a binary black hole waveform.  We first establish a
# dictionary of parameters that includes all of the different waveform
# parameters, including masses of the two black holes (mass_1, mass_2),
# spins of both black holes (a, tilt, phi), etc.
injection_parameters = dict(
    mass_1=36., mass_2=29., a_1=0.4, a_2=0.3, tilt_1=0.5, tilt_2=1.0,
    phi_12=1.7, phi_jl=0.3, luminosity_distance=2000., theta_jn=0.4, psi=2.659,
    phase=1.3, geocent_time=1126259642.413, ra=1.375, dec=-1.2108)

### Create a waveform generator

In [4]:
# Fixed arguments passed into the source model
waveform_arguments = dict(waveform_approximant='IMRPhenomPv2',
                          reference_frequency=50., minimum_frequency=20.)

# Create the waveform_generator using a LAL BinaryBlackHole source function
# the generator will convert all the parameters
waveform_generator = bilby.gw.WaveformGenerator(
    duration=duration, sampling_frequency=sampling_frequency,
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=waveform_arguments)

# Set up interferometers.  In this case we'll use two interferometers
# (LIGO-Hanford (H1), LIGO-Livingston (L1). These default to their design
# sensitivity
ifos = bilby.gw.detector.InterferometerList(['H1', 'L1'])
ifos.set_strain_data_from_power_spectral_densities(
    sampling_frequency=sampling_frequency, duration=duration,
    start_time=injection_parameters['geocent_time'] - 3)

ifos.inject_signal(waveform_generator=waveform_generator,
                   parameters=injection_parameters)

22:31 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters
22:31 bilby INFO    : Injected signal in H1:
22:31 bilby INFO    :   optimal SNR = 11.78
22:31 bilby INFO    :   matched filter SNR = 12.82-2.04j
22:31 bilby INFO    :   mass_1 = 36.0
22:31 bilby INFO    :   mass_2 = 29.0
22:31 bilby INFO    :   a_1 = 0.4
22:31 bilby INFO    :   a_2 = 0.3
22:31 bilby INFO    :   tilt_1 = 0.5
22:31 bilby INFO    :   tilt_2 = 1.0
22:31 bilby INFO    :   phi_12 = 1.7
22:31 bilby INFO    :   phi_jl = 0.3
22:31 bilby INFO    :   luminosity_distance = 2000.0
22:31 bilby INFO    :   theta_jn = 0.4
22:31 bilby INFO    :   psi = 2.659
22:31 bilby INFO    :   phase = 1.3
22:31 bilby INFO    :   geocent_time = 1126259642.413
22:31 bilby INFO    :   ra = 1.375
22:31 bilby INFO    :   dec = -1.2108
22:31 bilby INFO    : I

[{'plus': array([0.-0.j, 0.-0.j, 0.-0.j, ..., 0.-0.j, 0.-0.j, 0.-0.j]),
  'cross': array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j])},
 {'plus': array([0.-0.j, 0.-0.j, 0.-0.j, ..., 0.-0.j, 0.-0.j, 0.-0.j]),
  'cross': array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j])}]

### Create a prior

In [5]:
# For this analysis, we implement the standard BBH priors defined, except for
# the definition of the time prior, which is defined as uniform about the
# injected value.
# Furthermore, we decide to sample in chirp mass and mass ratio, due to the
# preferred shape for the associated posterior distributions.
priors = bilby.gw.prior.BBHPriorDict()
priors.pop('mass_1')
priors.pop('mass_2')

priors['chirp_mass'] = bilby.prior.Uniform(
    name='chirp_mass', latex_label='$M$', minimum=10.0, maximum=100.0,
    unit='$M_{\\odot}$')

priors['mass_ratio'] = bilby.prior.Uniform(
    name='mass_ratio', latex_label='$q$', minimum=0.5, maximum=1.0)

priors['geocent_time'] = bilby.core.prior.Uniform(
    minimum=injection_parameters['geocent_time'] - 0.1,
    maximum=injection_parameters['geocent_time'] + 0.1,
    name='geocent_time', latex_label='$t_c$', unit='$s$')

22:31 bilby INFO    : No prior given, using default BBH priors in /Users/dhruv/miniconda3/envs/igwn-py39/lib/python3.9/site-packages/bilby/gw/prior_files/precessing_spins_bbh.prior.


### Create a likelihood

In [6]:
# Initialise the likelihood by passing in the interferometer data (ifos) and
# the waveoform generator, as well the priors.
# The explicit time, distance, and phase marginalizations are turned on to
# improve convergence, and the parameters are recovered by the conversion
# function.
likelihood = bilby.gw.GravitationalWaveTransient(
    interferometers=ifos, waveform_generator=waveform_generator, priors=priors,
    distance_marginalization=True, phase_marginalization=True, time_marginalization=True)

22:32 bilby INFO    : Loaded distance marginalisation lookup table from .distance_marginalization_lookup.npz.


In [ ]:
# Run sampler. In this case we're going to use the `cpnest` sampler
# Note that the maxmcmc parameter is increased so that between each iteration of
# the nested sampler approach, the walkers will move further using an mcmc
# approach, searching the full parameter space.
# The conversion function will determine the distance, phase and coalescence
# time posteriors in post processing.
result = bilby.run_sampler(
    likelihood=likelihood, priors=priors, sampler='dynesty', npoints=2000,
    injection_parameters=injection_parameters, outdir=outdir,
    label=label, maxmcmc=2000,
    conversion_function=bilby.gw.conversion.generate_all_bbh_parameters)

# Make a corner plot.
result.plot_corner()

12:29 bilby INFO    : Running for label 'full_15_parameters', output will be saved to 'outdir'
12:29 bilby INFO    : Using lal version 7.2.4
12:29 bilby INFO    : Using lal git version Branch: None;Tag: lalsuite-v7.11;Id: 7a2f2aa176ad39eeaede38f6df4a41d6bf226e8f;;Builder: Unknown User <>;Repository status: CLEAN: All modifications committed
12:29 bilby INFO    : Using lalsimulation version 4.0.2
12:29 bilby INFO    : Using lalsimulation git version Branch: None;Tag: lalsuite-v7.11;Id: 7a2f2aa176ad39eeaede38f6df4a41d6bf226e8f;;Builder: Unknown User <>;Repository status: CLEAN: All modifications committed
12:29 bilby INFO    : Analysis priors:
12:29 bilby INFO    : mass_ratio=Uniform(minimum=0.5, maximum=1.0, name='mass_ratio', latex_label='$q$', unit=None, boundary=None)
12:29 bilby INFO    : chirp_mass=Uniform(minimum=10.0, maximum=100.0, name='chirp_mass', latex_label='$M$', unit='$M_{\\odot}$', boundary=None)
12:29 bilby INFO    : dec=Cosine(minimum=-1.5707963267948966, maximum=1.570

1it [00:00, ?it/s]

12:40 bilby INFO    : Written checkpoint file outdir/full_15_parameters_resume.pickle
12:51 bilby INFO    : Written checkpoint file outdir/full_15_parameters_resume.pickle
13:01 bilby INFO    : Written checkpoint file outdir/full_15_parameters_resume.pickle
13:12 bilby INFO    : Written checkpoint file outdir/full_15_parameters_resume.pickle
13:24 bilby INFO    : Written checkpoint file outdir/full_15_parameters_resume.pickle
13:36 bilby INFO    : Written checkpoint file outdir/full_15_parameters_resume.pickle
13:47 bilby INFO    : Written checkpoint file outdir/full_15_parameters_resume.pickle
13:58 bilby INFO    : Written checkpoint file outdir/full_15_parameters_resume.pickle
14:09 bilby INFO    : Written checkpoint file outdir/full_15_parameters_resume.pickle
14:19 bilby INFO    : Written checkpoint file outdir/full_15_parameters_resume.pickle
14:30 bilby INFO    : Written checkpoint file outdir/full_15_parameters_resume.pickle
14:41 bilby INFO    : Written checkpoint file outdir/f

21:26 bilby INFO    : Written checkpoint file outdir/full_15_parameters_resume.pickle
21:36 bilby INFO    : Written checkpoint file outdir/full_15_parameters_resume.pickle


In [11]:
samples = result.posterior

In [12]:
samples

mass_ratio  chirp_mass       a_1       a_2    tilt_1    tilt_2    phi_12   
0      0.836028   31.725626  0.662000  0.645164  1.615677  1.726965  4.926762  \
1      0.875399   31.063765  0.132188  0.105331  1.358270  1.928139  0.768192   
2      0.999707   30.428421  0.920642  0.382177  2.155972  0.662301  4.522789   
3      0.859521   32.893418  0.125905  0.289173  0.897341  1.349229  6.121069   
4      0.558228   31.036358  0.316932  0.047103  1.311152  1.040560  4.302064   
..          ...         ...       ...       ...       ...       ...       ...   
525    0.992363   31.309936  0.163289  0.437566  0.156190  1.952186  1.435859   
526    0.896271   31.112798  0.461017  0.397027  1.125093  2.281037  4.242951   
527    0.880654   30.833658  0.752558  0.875270  2.071190  1.219948  4.574085   
528    0.991459   30.882850  0.270110  0.941317  3.016109  1.445538  2.010849   
529    0.894649   30.420744  0.706310  0.841501  0.704509  2.765511  2.406512   

       phi_jl       dec        ra  ...  chi_2_in_plane     chi_p  cos_tilt_1   
0    4.825046 -0.967467  2.555441  ...        0.637312  0.661333   -0.044866  \
1    2.507927 -1.311102  1.566008  ...        0.098677  0.129214    0.210930   
2    3.088706 -0.739912  2.712823  ...        0.235013  0.767461   -0.552346   
3    0.280472 -1.119265  2.255562  ...        0.282104  0.237297    0.623690   
4    5.536851 -1.277885  1.596640  ...        0.040636  0.306309    0.256736   
..        ...       ...       ...  ...             ...       ...         ...   
525  6.124195 -1.270721  2.015710  ...        0.406126  0.402584    0.987827   
526  3.958598 -1.270887  1.620534  ...        0.301028  0.415980    0.431092   
527  3.781005 -1.247400  2.172724  ...        0.821950  0.710847   -0.479771   
528  1.798495 -1.244329  2.093363  ...        0.933943  0.924832   -0.992137   
529  1.746670 -1.268648  1.852380  ...        0.309065  0.457448    0.761930   

     cos_tilt_2  redshift  comoving_distance  mass_1_source  mass_2_source   
0     -0.155535  0.117237         504.464714      35.703288      29.848952  \
1     -0.349786  0.075153         326.726022      35.487797      31.065973   
2      0.788580  0.108361         467.292231      31.540449      31.531218   
3      0.219759  0.111736         481.448616      36.680348      31.527542   
4      0.505737  0.096663         418.045661      43.876976      24.493373   
..          ...       ...                ...            ...            ...   
525   -0.372210  0.118259         508.734199      32.285758      32.039202   
526   -0.652016  0.110091         474.550314      34.017081      30.488530   
527    0.343695  0.109967         474.030173      34.016824      29.957046   
528    0.124931  0.109807         473.361294      32.102534      31.828341   
529   -0.930111  0.115067         495.393151      33.142324      29.650747   

     chirp_mass_source total_mass_source  
0            28.396504         65.552239  
1            28.892417         66.553770  
2            27.453537         63.071667  
3            29.587428         68.207890  
4            28.300728         68.370348  
..                 ...               ...  
525          27.998818         64.324960  
526          28.027258         64.505611  
527          27.778905         63.973870  
528          27.827223         63.930875  
529          27.281536         62.793071  

[530 rows x 52 columns]

In [13]:
type(samples)

pandas.core.frame.DataFrame

In [16]:
samples.columns.tolist()[:13]

['mass_ratio',
 'chirp_mass',
 'a_1',
 'a_2',
 'tilt_1',
 'tilt_2',
 'phi_12',
 'phi_jl',
 'dec',
 'ra',
 'theta_jn',
 'psi',
 'time_jitter']